In [1]:
import torch
import torch.nn.functional as F
import numpy as np
import uproot
import awkward as ak
import concurrent.futures
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, ConcatDataset, random_split
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import torch_geometric
from sklearn.metrics import roc_auc_score, roc_curve

In [2]:
class ParticleStaticEdgeConv(torch_geometric.nn.MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(ParticleStaticEdgeConv, self).__init__(aggr='mean')
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(2 * in_channels, out_channels[0], bias=False),
            torch_geometric.nn.BatchNorm(out_channels[0]), 
            torch.nn.ReLU(),
            torch.nn.Linear(out_channels[0], out_channels[1], bias=False),
            torch_geometric.nn.BatchNorm(out_channels[1]),
            torch.nn.ReLU(),
            torch.nn.Linear(out_channels[1], out_channels[2], bias=False),
            torch_geometric.nn.BatchNorm(out_channels[2]),
            torch.nn.ReLU()
        )

    def forward(self, x, edge_index, k):
        
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)

    def message(self, edge_index, x_i, x_j):
        tmp = torch.cat([x_i, x_j - x_i], dim = 1)

        out_mlp = self.mlp(tmp)

        return out_mlp

    def update(self, aggr_out):
        return aggr_out

class ParticleDynamicEdgeConv(ParticleStaticEdgeConv):
    def __init__(self, in_channels, out_channels, k=7):
        super(ParticleDynamicEdgeConv, self).__init__(in_channels, out_channels)
        self.k = k
        self.skip_mlp = torch.nn.Sequential(
            torch.nn.Linear(in_channels, out_channels[2], bias=False),
            torch_geometric.nn.BatchNorm(out_channels[2]),
        )
        self.act = torch.nn.ReLU()

    def forward(self, pts, fts, batch=None):
        edges = torch_geometric.nn.knn_graph(pts, self.k, batch, loop=False, flow=self.flow)
        aggrg = super(ParticleDynamicEdgeConv, self).forward(fts, edges, self.k)
        x = self.skip_mlp(fts)
        out = torch.add(aggrg, x)
        return self.act(out)


class ParticleNet(torch.nn.Module):

    def __init__(self, settings):
        super().__init__()
        previous_output_shape = settings['input_features']

        self.input_bn = torch_geometric.nn.BatchNorm(settings['input_features'])

        self.conv_process = torch.nn.ModuleList()
        for layer_idx, layer_param in enumerate(settings['conv_params']):
            K, channels = layer_param
            self.conv_process.append(ParticleDynamicEdgeConv(previous_output_shape, channels, k=K).to(DEVICE))
            previous_output_shape = channels[-1]



        self.fc_process = torch.nn.ModuleList()
        for layer_idx, layer_param in enumerate(settings['fc_params']):
            drop_rate, units = layer_param
            seq = torch.nn.Sequential(
                torch.nn.Linear(previous_output_shape, units),
                torch.nn.Dropout(p=drop_rate),
                torch.nn.ReLU()
            ).to(DEVICE)
            self.fc_process.append(seq)
            previous_output_shape = units


        self.output_mlp_linear = torch.nn.Linear(previous_output_shape, settings['output_classes'])
        self.output_activation = torch.nn.Softmax(dim=0)

    def forward(self, batch):
        fts = self.input_bn(batch.x)
        pts = batch.pos

        for idx, layer in enumerate(self.conv_process):
          fts = layer(pts, fts, batch.batch)
          pts = fts

        x = torch_geometric.nn.global_mean_pool(fts, batch.batch)

        for layer in self.fc_process:
            x = layer(x)

        x = self.output_mlp_linear(x)
        x = self.output_activation(x)
        return x

In [3]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
# DEVICE = "cpu"
print(DEVICE)
settings = {
    "conv_params": [
        (16, (64, 64, 64)),
        (16, (128, 128, 128)),
        (16, (256, 256, 256)),
    ],
    "fc_params": [
        (0.1, 256)
    ],
    "input_features": 17,
    "output_classes": 1,
}

model = ParticleNet(settings)
print(model)

cuda
ParticleNet(
  (input_bn): BatchNorm(17)
  (conv_process): ModuleList(
    (0-2): 3 x ParticleDynamicEdgeConv()
  )
  (fc_process): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=256, out_features=256, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): ReLU()
    )
  )
  (output_mlp_linear): Linear(in_features=256, out_features=1, bias=True)
  (output_activation): Softmax(dim=0)
)


In [4]:
if DEVICE == 'cuda':
    gpu_name = torch.cuda.get_device_name(0)
    print("GPU Name:", gpu_name)

GPU Name: NVIDIA GeForce RTX 3090


In [5]:
fileset = {}

sig_dir = '/mdsmlvol/rechits_v4/point_clouds/'
fileset['sample'] = [sig_dir + f'BToKPhi_MuonLLPDecayGenFilter_PhiToPi0Pi0_mPhi0p3_ctau300_{str(i).zfill(7)}_point_clouds.pt' for i in range(328)]
# fileset['sample'] = [sig_dir + f'BToKPhi_MuonLLPDecayGenFilter_PhiToPi0Pi0_mPhi0p3_ctau300_{str(i).zfill(7)}_point_clouds.pt' for i in range(10)]

# bkg_dir = '/ceph/cms/store/user/aaportel/B-Parking/rechits_v2/ParkingBPH1_2018A/'
# fileset['background'] = [bkg_dir + f'ParkingBPH1_2018A_{str(i).zfill(7)}.root' for i in range(380)]

In [7]:
shuffle_dataset = True
batch_size = 64
random_seed = 42
split_ratio = 0.8  # 80% of the data for training

# Assuming an even split of the remaining 20% for test and validation
test_validation_split_ratio = 0.5  # Split the remaining 20% evenly into test and validation

# Set random seed for reproducibility
torch.manual_seed(random_seed)

# Assuming 'fileset' and 'torch.load' are defined elsewhere in your code
datasets = [torch.load(fp) for fp in fileset['sample']]
dataset = ConcatDataset(datasets)

dataset_size = len(dataset)
train_size = int(split_ratio * dataset_size)
test_validation_size = dataset_size - train_size
test_size = int(test_validation_size * test_validation_split_ratio)
validation_size = test_validation_size - test_size

# Split the dataset into train, test, and validation sets
train_dataset, temp_test_validation_dataset = random_split(dataset, [train_size, test_validation_size])
test_dataset, validation_dataset = random_split(temp_test_validation_dataset, [test_size, validation_size])

# Create data loaders for each set
train_loader = DataLoader(train_dataset, batch_size, shuffle=shuffle_dataset)
test_loader = DataLoader(test_dataset, batch_size, shuffle=shuffle_dataset)
validation_loader = DataLoader(validation_dataset, batch_size, shuffle=shuffle_dataset)

In [80]:
import torch
import torch.nn.functional as F
from torch.optim.lr_scheduler import OneCycleLR
from torch.optim.lr_scheduler import LambdaLR


# Define the training loop with scheduler step
def train(model, device, train_loader, optimizer, scheduler):
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)  # Ensure your model outputs are correctly aligned with the targets
        loss = F.binary_cross_entropy(output, data.y.view(-1, 1).type_as(output))  # Ensure this matches your data structure
        loss.backward()
        optimizer.step()
        scheduler.step()  # Step the scheduler after each batch
        total_loss += loss.item()
    return total_loss / len(train_loader)

# Define the evaluation/testing loop
def evaluate(model, device, loader):
    model.eval()
    targets = []
    outputs = []

    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            output = model(data)
            targets.extend(data.y.view(-1, 1).cpu().numpy())  # Ensure this matches your data structure
            outputs.extend(output.cpu().numpy())

    return targets, outputs

# Define the validation loop
def validate(model, device, validation_loader):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for data in validation_loader:
            data = data.to(device)
            output = model(data)
            loss = F.binary_cross_entropy(output, data.y.view(-1, 1).type_as(output))  # Ensure this matches your data structure
            total_loss += loss.item()
    return total_loss / len(validation_loader)

# Setup optimizer with weight decay
model = ParticleNet(settings)
model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1, weight_decay=0.0001)

epochs = 21
# Setup LambdaLR scheduler with the parameters according to the documentation
def lr_lambda(epoch):
    initial_lr = 3e-4
    max_lr = 3e-3
    min_lr = 5e-7
    lr = 0
    if epoch < 9:
        lr = epoch/8 * (max_lr - initial_lr) + initial_lr
        # print(lr)
        return lr
    elif (epoch >= 9 and epoch < 17):
        return -(epoch-8)/8 * (max_lr - initial_lr) + max_lr
    else: 
        return -(epoch-16)/4 * (initial_lr - min_lr) + initial_lr

scheduler = LambdaLR(optimizer, lr_lambda=lr_lambda)

# Initialize lists to track the losses
train_losses = []
validation_losses = []

# Training loop
for epoch in range(epochs):
    train_loss = train(model, DEVICE, train_loader, optimizer, scheduler)  # Pass scheduler here
    validation_loss = validate(model, DEVICE, validation_loader)
    
    # Record the losses for plotting
    train_losses.append(train_loss)
    validation_losses.append(validation_loss)

    
    learningrate = scheduler.get_last_lr()[0]
    print("LR" + learningrate)
    print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Validation Loss: {validation_loss:.4f}')

KeyboardInterrupt: 

In [ ]:
# Evaluation on the test set
targets, outputs = evaluate(model, DEVICE, validation_loader)

# Compute ROC curve and ROC area
targets = np.array(targets)
outputs = np.array(outputs)
fpr, tpr, _ = roc_curve(targets, outputs)
roc_auc = roc_auc_score(targets, outputs)

# Plotting ROC Curve
# plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='Particle Net (area = %0.2f)' % roc_auc)

import torch
from torch_geometric.data import Data
import uproot
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt
# import concurrent.futures

fileset = {}

sig_dir = '/mdsmlvol/rechits_v4/raw/'
fileset['sample'] = [sig_dir + f'BToKPhi_MuonLLPDecayGenFilter_PhiToPi0Pi0_mPhi0p3_ctau300_{str(i).zfill(7)}.root' for i in range(328)]
# fileset['sample'] = [sig_dir + f'BToKPhi_MuonLLPDecayGenFilter_PhiToPi0Pi0_mPhi0p3_ctau300_{str(i).zfill(7)}.root' for i in range(2)]

# bkg_dir = '/ceph/cms/store/user/aaportel/B-Parking/rechits_v2/ParkingBPH1_2018A/'
# fileset['background'] = [bkg_dir + f'ParkingBPH1_2018A_{str(i).zfill(7)}.root' for i in range(380)]

size_arrays = []
label_arrays = []

for file in fileset['sample']:
    f = uproot.open(file)
    clusterSize = f['MuonSystem:cscRechitClusterSize'].arrays(library = 'ak')['cscRechitClusterSize']
    clusterEta = f['MuonSystem:cscRechitClusterEta'].arrays(library = 'ak')['cscRechitClusterEta']
    clusterPhi = f['MuonSystem:cscRechitClusterPhi'].arrays(library = 'ak')['cscRechitClusterPhi']

    gLLP_eta = f['MuonSystem:gLLP_eta'].arrays(library = 'ak')['gLLP_eta']
    gLLP_phi = f['MuonSystem:gLLP_phi'].arrays(library = 'ak')['gLLP_phi']

    # ids = raw_data.cscRechitClusterSegmentID[raw_data.cscRechitClusterSegmentID!=-999]

    # def clusterer(data, ids):
    #     clustered = []
    #     for i in range(len(ids)):
    #         clusters = []
    #         for id in ids[i]:
    #             clusters.append(data[i][id])
    #         clustered.append(clusters)    
    #     return ak.flatten(ak.Array(clustered), axis = 1)
    
    # cscRechits_clustered = ak.zip({
    #     "Phi": clusterer(raw_data.cscRechitsPhi, ids),
    #     "Eta": clusterer(raw_data.cscRechitsEta, ids),
    #     "X": clusterer(raw_data.cscRechitsX, ids),
    #     "Y": clusterer(raw_data.cscRechitsY, ids),
    #     "Z": clusterer(raw_data.cscRechitsZ, ids),
    #     "E": clusterer(raw_data.cscRechitsE, ids),
    #     "Tpeak": clusterer(raw_data.cscRechitsTpeak, ids),
    #     "Twire": clusterer(raw_data.cscRechitsTwire, ids),
    #     "Quality": clusterer(raw_data.cscRechitsQuality, ids),
    #     "Chamber": clusterer(raw_data.cscRechitsChamber, ids),
    #     "Station": clusterer(raw_data.cscRechitsStation, ids),
    #     "Channels": clusterer(raw_data.cscRechitsChannels, ids),
    #     "NStrips": clusterer(raw_data.cscRechitsNStrips, ids),
    #     "HitWire": clusterer(raw_data.cscRechitsHitWire, ids),
    #     "WGroupsBX": clusterer(raw_data.cscRechitsWGroupsBX, ids),
    #     "NWireGroups": clusterer(raw_data.cscRechitsNWireGroups, ids),
    #     "DetId": clusterer(raw_data.cscRechitsDetId, ids),
    # })
    
    # cscRechitClusters = ak.zip({
    #     "X": ak.flatten(raw_data.cscRechitClusterX, axis = 1),
    #     "Y": ak.flatten(raw_data.cscRechitClusterY, axis = 1),
    #     "Z": ak.flatten(raw_data.cscRechitClusterZ, axis = 1),
    #     "Time": ak.flatten(raw_data.cscRechitClusterTime, axis = 1),
    #     "TimeWeighted": ak.flatten(raw_data.cscRechitClusterTimeWeighted, axis = 1),
    #     "TimeTotal": ak.flatten(raw_data.cscRechitClusterTimeTotal, axis = 1),
    #     "TimeSpread": ak.flatten(raw_data.cscRechitClusterTimeSpread, axis = 1),
    #     "TimeSpreadWeighted": ak.flatten(raw_data.cscRechitClusterTimeSpreadWeighted, axis = 1),
    #     "TimeSpreadWeightedAll": ak.flatten(raw_data.cscRechitClusterTimeSpreadWeightedAll, axis = 1),
    #     "GenMuonDeltaR": ak.flatten(raw_data.cscRechitClusterGenMuonDeltaR, axis = 1),
    #     "Phi": ak.flatten(raw_data.cscRechitClusterPhi, axis = 1),
    #     "Eta": ak.flatten(raw_data.cscRechitClusterEta, axis = 1),
    #     "Size": ak.flatten(raw_data.cscRechitClusterSize, axis = 1),
    #     "Me11Ratio": ak.flatten(raw_data.cscRechitClusterMe11Ratio, axis = 1),
    #     "Me12Ratio": ak.flatten(raw_data.cscRechitClusterMe12Ratio, axis = 1),
    #     "NStation": ak.flatten(raw_data.cscRechitClusterNStation, axis = 1),
    #     "NStation5": ak.flatten(raw_data.cscRechitClusterNStation5, axis = 1),
    #     "NStation10": ak.flatten(raw_data.cscRechitClusterNStation10, axis = 1),
    #     "NStation10perc": ak.flatten(raw_data.cscRechitClusterNStation10perc, axis = 1),
    #     "AvgStation": ak.flatten(raw_data.cscRechitClusterAvgStation, axis = 1),
    #     "AvgStation5": ak.flatten(raw_data.cscRechitClusterAvgStation5, axis = 1),
    #     "AvgStation10": ak.flatten(raw_data.cscRechitClusterAvgStation10, axis = 1),
    #     "AvgStation10perc": ak.flatten(raw_data.cscRechitClusterAvgStation10perc, axis = 1),
    #     "MatchedTrackSumPt_0p2": ak.flatten(raw_data.cscRechitClusterMatchedTrackSumPt_0p2, axis = 1),
    #     "MatchedTrackLeadPt_0p2": ak.flatten(raw_data.cscRechitClusterMatchedTrackLeadPt_0p2, axis = 1),
    #     "MatchedTrackSize_0p2": ak.flatten(raw_data.cscRechitClusterMatchedTrackSize_0p2, axis = 1),
    #     "MatchedTrackSumPt_0p3": ak.flatten(raw_data.cscRechitClusterMatchedTrackSumPt_0p3, axis = 1),
    #     "MatchedTrackLeadPt_0p3": ak.flatten(raw_data.cscRechitClusterMatchedTrackLeadPt_0p3, axis = 1),
    #     "MatchedTrackSize_0p3": ak.flatten(raw_data.cscRechitClusterMatchedTrackSize_0p3, axis = 1),
    #     "MatchedTrackSumPt_0p4": ak.flatten(raw_data.cscRechitClusterMatchedTrackSumPt_0p4, axis = 1),
    #     "MatchedTrackLeadPt_0p4": ak.flatten(raw_data.cscRechitClusterMatchedTrackLeadPt_0p4, axis = 1),
    #     "MatchedTrackSize_0p4": ak.flatten(raw_data.cscRechitClusterMatchedTrackSize_0p4, axis = 1),
    #     "MatchedTrackSumPt_trk_pos_0p5": ak.flatten(raw_data.cscRechitClusterMatchedTrackSumPt_trk_pos_0p5, axis = 1),
    #     "MatchedTrackLeadPt_trk_pos_0p5": ak.flatten(raw_data.cscRechitClusterMatchedTrackLeadPt_trk_pos_0p5, axis = 1),
    #     "MatchedTrackSize_trk_pos_0p5": ak.flatten(raw_data.cscRechitClusterMatchedTrackSize_trk_pos_0p5, axis = 1),
    #     "MatchedTrackSumPt_trk_pos_0p2": ak.flatten(raw_data.cscRechitClusterMatchedTrackSumPt_trk_pos_0p2, axis = 1),
    #     "MatchedTrackLeadPt_trk_pos_0p2": ak.flatten(raw_data.cscRechitClusterMatchedTrackLeadPt_trk_pos_0p2, axis = 1),
    #     "MatchedTrackSize_trk_pos_0p2": ak.flatten(raw_data.cscRechitClusterMatchedTrackSize_trk_pos_0p2, axis = 1),
    #     "MatchedTrackSumPt_trk_pos_0p3": ak.flatten(raw_data.cscRechitClusterMatchedTrackSumPt_trk_pos_0p3, axis = 1),
    #     "MatchedTrackLeadPt_trk_pos_0p3": ak.flatten(raw_data.cscRechitClusterMatchedTrackLeadPt_trk_pos_0p3, axis = 1),
    #     "MatchedTrackSize_trk_pos_0p3": ak.flatten(raw_data.cscRechitClusterMatchedTrackSize_trk_pos_0p3, axis = 1),
    #     "MatchedTrackSumPt_trk_pos_0p4": ak.flatten(raw_data.cscRechitClusterMatchedTrackSumPt_trk_pos_0p4, axis = 1),
    #     "MatchedTrackLeadPt_trk_pos_0p4": ak.flatten(raw_data.cscRechitClusterMatchedTrackLeadPt_trk_pos_0p4, axis = 1),
    #     "MatchedTrackSize_trk_pos_0p4": ak.flatten(raw_data.cscRechitClusterMatchedTrackSize_trk_pos_0p4, axis = 1),
    #     "MaxStation": ak.flatten(raw_data.cscRechitClusterMaxStation, axis = 1),
    #     "MaxStationRatio": ak.flatten(raw_data.cscRechitClusterMaxStationRatio, axis = 1),
    #     "NChamber": ak.flatten(raw_data.cscRechitClusterNChamber, axis = 1),
    #     "MaxChamber": ak.flatten(raw_data.cscRechitClusterMaxChamber, axis = 1),
    #     "MaxChamberRatio": ak.flatten(raw_data.cscRechitClusterMaxChamberRatio, axis = 1),
    #     "NRechitChamberPlus11": ak.flatten(raw_data.cscRechitClusterNRechitChamberPlus11, axis = 1),
    #     "NRechitChamberPlus12": ak.flatten(raw_data.cscRechitClusterNRechitChamberPlus12, axis = 1),
    #     "NRechitChamberPlus13": ak.flatten(raw_data.cscRechitClusterNRechitChamberPlus13, axis = 1),
    #     "NRechitChamberPlus21": ak.flatten(raw_data.cscRechitClusterNRechitChamberPlus21, axis = 1),
    #     "NRechitChamberPlus22": ak.flatten(raw_data.cscRechitClusterNRechitChamberPlus22, axis = 1),
    #     "NRechitChamberPlus31": ak.flatten(raw_data.cscRechitClusterNRechitChamberPlus31, axis = 1),
    #     "NRechitChamberPlus32": ak.flatten(raw_data.cscRechitClusterNRechitChamberPlus32, axis = 1),
    #     "NRechitChamberPlus41": ak.flatten(raw_data.cscRechitClusterNRechitChamberPlus41, axis = 1),
    #     "NRechitChamberPlus42": ak.flatten(raw_data.cscRechitClusterNRechitChamberPlus42, axis = 1),
    #     "NRechitChamberMinus11": ak.flatten(raw_data.cscRechitClusterNRechitChamberMinus11, axis = 1),
    #     "NRechitChamberMinus12": ak.flatten(raw_data.cscRechitClusterNRechitChamberMinus12, axis = 1),
    #     "NRechitChamberMinus13": ak.flatten(raw_data.cscRechitClusterNRechitChamberMinus13, axis = 1),
    #     "NRechitChamberMinus21": ak.flatten(raw_data.cscRechitClusterNRechitChamberMinus21, axis = 1),
    #     "NRechitChamberMinus22": ak.flatten(raw_data.cscRechitClusterNRechitChamberMinus22, axis = 1),
    #     "NRechitChamberMinus31": ak.flatten(raw_data.cscRechitClusterNRechitChamberMinus31, axis = 1),
    #     "NRechitChamberMinus32": ak.flatten(raw_data.cscRechitClusterNRechitChamberMinus32, axis = 1),
    #     "NRechitChamberMinus41": ak.flatten(raw_data.cscRechitClusterNRechitChamberMinus41, axis = 1),
    #     "NRechitChamberMinus42": ak.flatten(raw_data.cscRechitClusterNRechitChamberMinus42, axis = 1),
    #     "_match_dtSeg_0p4": ak.flatten(raw_data.cscRechitCluster_match_dtSeg_0p4, axis = 1),
    #     "_match_MB1Seg_0p4": ak.flatten(raw_data.cscRechitCluster_match_MB1Seg_0p4, axis = 1),
    #     "_match_RB1_0p4": ak.flatten(raw_data.cscRechitCluster_match_RB1_0p4, axis = 1),
    #     "_match_RE12_0p4": ak.flatten(raw_data.cscRechitCluster_match_RE12_0p4, axis = 1),
    #     "_match_gLLP_deltaR": ak.flatten(raw_data.cscRechitCluster_match_gLLP_deltaR, axis = 1),
    #     "JetVetoPt": ak.flatten(raw_data.cscRechitClusterJetVetoPt, axis = 1),
    #     "MuonVetoPt": ak.flatten(raw_data.cscRechitClusterMuonVetoPt, axis = 1),
    #     "GenMuonVetoPt_dR0p8": ak.flatten(raw_data.cscRechitClusterGenMuonVetoPt_dR0p8, axis = 1),
    #     "GenMuonVetoPt": ak.flatten(raw_data.cscRechitClusterGenMuonVetoPt, axis = 1),
    #     "MetEENoise_dPhi": ak.flatten(raw_data.cscRechitClusterMetEENoise_dPhi, axis = 1),
        
    # })

    def deltaR(eta1, phi1, eta2, phi2):
        deta = eta1 - eta2
        dphi = np.arctan2(np.sin(phi1 - phi2), np.cos(phi1 - phi2))
        return np.sqrt(deta**2 + dphi**2)
    
    labels = ak.flatten(
            deltaR(clusterEta, clusterPhi, gLLP_eta, gLLP_phi) < .8, 
        axis = 1)

    size_array_current = ak.flatten(clusterSize, axis=1)
    size_arrays.append(size_array_current)
    label_arrays.append(labels)

# After the loop, concatenate all Size arrays into one
aggregated_size_array = ak.concatenate(size_arrays, axis=0)
aggregated_label_array = ak.concatenate(label_arrays, axis=0)

import numpy as np
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Step 2: Generate ROC curve data
fpr, tpr, thresholds = roc_curve(aggregated_label_array, aggregated_size_array)
roc_auc = auc(fpr, tpr)

# Step 3: Plot the ROC curve
plt.plot(fpr, tpr, color='green', label='Cluster Size Discriminator (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()